# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921686382259                   -0.69    5.9         
  2   -7.926162735519       -2.35       -1.22    1.0    258ms
  3   -7.926836966102       -3.17       -2.37    1.6    230ms
  4   -7.926861503672       -4.61       -3.02    2.8    245ms
  5   -7.926861630292       -6.90       -3.34    1.9    218ms
  6   -7.926861666290       -7.44       -3.71    1.5    240ms
  7   -7.926861680658       -7.84       -4.40    1.1    204ms
  8   -7.926861681803       -8.94       -4.91    2.1    230ms
  9   -7.926861681867      -10.20       -5.43    1.9    220ms
 10   -7.926861681872      -11.28       -6.11    2.0    249ms
 11   -7.926861681873      -12.32       -6.60    2.2    240ms
 12   -7.926861681873      -13.39       -7.33    1.9    227ms
 13   -7.926861681873      -14.57       -7.93    2.4    255ms
 14   -7.926861681873      -14.45       -7.91    2.5    230ms
 15   -7.

If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921705181704                   -0.69    5.9         
  2   -7.926167135204       -2.35       -1.22    1.0    212ms
  3   -7.926836884198       -3.17       -2.37    1.8    287ms
  4   -7.926861454507       -4.61       -3.01    2.6    247ms
  5   -7.926861630819       -6.75       -3.33    1.9    218ms
  6   -7.926861665901       -7.45       -3.71    1.5    206ms
  7   -7.926861680462       -7.84       -4.37    1.1    233ms
  8   -7.926861681796       -8.87       -4.91    2.1    224ms
  9   -7.926861681858      -10.21       -5.20    2.1    223ms
 10   -7.926861681872      -10.88       -6.00    1.6    223ms
 11   -7.926861681873      -12.01       -6.94    2.4    241ms
 12   -7.926861681873      -13.67       -7.32    2.8    252ms
 13   -7.926861681873   +    -Inf       -8.01    1.6    212ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921663465105                   -0.69    5.9         
  2   -7.926161831559       -2.35       -1.22    1.0    269ms
  3   -7.926839758831       -3.17       -2.37    1.8    250ms
  4   -7.926864914630       -4.60       -3.01    2.8    275ms
  5   -7.926865037532       -6.91       -3.32    1.6    222ms
  6   -7.926865076955       -7.40       -3.71    1.5    254ms
  7   -7.926865091780       -7.83       -4.42    1.5    211ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
